#Example hologram processing pipeline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import os

from pyopia.classify import Classify
import pyopia.process
import pyopia.io
import pyopia.background
import pyopia.statistics
import exampledata
from pyopia.pipeline import Pipeline
import pyopia.instrument.holo as holo

In [ ]:
#download the default holo test data and create file list
foldername = exampledata.get_folder_from_holo_repository()


In [ ]:
# example future pre-processing admin:
# 1) config file config load
# 2) build steps dict from loaded file
# 3) establish non-processing related metadata

foldername = 'holo_test_data_01'

os.makedirs('proc', exist_ok=True)

datafile_hdf = 'proc/holotest'
if os.path.exists(datafile_hdf + '-STATS.h5'):
  os.remove(datafile_hdf + '-STATS.h5')

model_path = exampledata.get_example_model()
threshold = 0.9

average_window = 10  # number of images to use as background

files = sorted(glob(os.path.join(foldername, '*.pgm')))
bgfiles = files[:average_window] 

holo_initial_settings = {'pixel_size': 4.4, # pixel size in um
                        'wavelength': 658, # laser wavelength in nm
                        'minZ': 27, # minimum reconstruction distance in mm
                        'maxZ': 65, # maximum reconstruction distance in mm
                        'stepZ': 0.5} #step size in mm

steps = {'initial': holo.Initial(files[0], **holo_initial_settings),
         'classifier': Classify(model_path=model_path),
         'create background': pyopia.background.CreateBackground(bgfiles,
                                                                 holo.load_image),
         'load': holo.Load(),
         'correct background': pyopia.background.CorrectBackgroundAccurate(pyopia.background.shift_bgstack_accurate),
         'reconstruct': holo.Reconstruct(stack_clean=0.02),
         'focus': holo.Focus(pyopia.instrument.holo.std_map,threshold=threshold,focus_function=pyopia.instrument.holo.find_focus_sobel),
         'segmentation': pyopia.process.Segment(threshold=threshold),
         'statextract': pyopia.process.CalculateStats(export_outputpath="proc"),
         'merge holo stats': holo.MergeStats(),
         'output': pyopia.io.StatsH5(datafile_hdf)
         }

processing_pipeline = Pipeline(steps)


In [ ]:
#do a test run with just the first few input files
for filename in files[:5]:
    stats = processing_pipeline.run(filename)

In [ ]:

# display metadata in the h5
pyopia.io.show_h5_meta(datafile_hdf + '-STATS.h5')

In [ ]:
# load the stats DataFrame from the h5 file
stats = pd.read_hdf(datafile_hdf + '-STATS.h5', 'ParticleStats/stats')
print('stats header: ', stats.columns)
print('Total number of particles: ', len(stats))
# Calculate the volume distribution from the stats DataFrame.
dias, vd = pyopia.statistics.vd_from_stats(stats, 24)

# plot the volume distribution
plt.style.use('dark_background')
plt.plot(dias, vd)
plt.xscale('log')
plt.xlabel('ECD [um]')
plt.ylabel('Volume Distribution [uL/sample vol.]')

In [ ]:
#plot histogram of focus locations
import numpy as np
plt.style.use('dark_background')
zval = np.arange(holo_initial_settings['minZ'], holo_initial_settings['maxZ'], holo_initial_settings['stepZ'])
plt.hist(zval[stats.ifocus-1],len(zval))
plt.xlim(zval[0],zval[-1])
plt.xlabel('Z [mm]')

In [ ]:
im_mont = pyopia.statistics.make_montage(datafile_hdf + '-STATS.h5',holo_initial_settings["pixel_size"],"proc",
    auto_scaler=1000, msize=2048, maxlength=100000, crop_stats=None)
plt.imshow(im_mont)